In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import os

### Import Images

In [2]:
def plot_image(img):
    fig = plt.figure(figsize=(10,8))
    plt.imshow(img, cmap='gray')
    plt.show()

In [1]:
def return_labeled_data_gray():
    TARGET_NO_MASK_DIR="../images_gray_transformed_no_mask"
    TARGET_MASK_DIR="../images_gray_transformed_mask"
    # no mask
    data_list = []
    target_list = []
    for filename in os.listdir(TARGET_NO_MASK_DIR):
        img = cv2.imread(f'{TARGET_NO_MASK_DIR}/{filename}', cv2.IMREAD_GRAYSCALE)
        data_list.append(img)
        target_list.append(0.0)
        
    # mask
    for filename in os.listdir(TARGET_MASK_DIR):
        img = cv2.imread(f'{TARGET_MASK_DIR}/{filename}', cv2.IMREAD_GRAYSCALE)
        data_list.append(img)
        target_list.append(1.0)
    return data_list, target_list

In [2]:
def return_labeled_data_color():
    TARGET_NO_MASK_DIR="../images_color_transformed_no_mask"
    TARGET_MASK_DIR="../images_color_transformed_mask"
    # no mask
    data_list = []
    target_list = []
    for filename in os.listdir(TARGET_NO_MASK_DIR):
        img = cv2.imread(f'{TARGET_NO_MASK_DIR}/{filename}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        data_list.append(img)
        target_list.append(0.0)
        
    # mask
    for filename in os.listdir(TARGET_MASK_DIR):
        img = cv2.imread(f'{TARGET_MASK_DIR}/{filename}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        data_list.append(img)
        target_list.append(1.0)
    return data_list, target_list

In [122]:
#  Grayscale Data
data_raw, target_raw = return_labeled_data_gray()
data_raw, target_raw = np.array(data_raw)/255, np.array(target_raw)

In [4]:
#  Color Data
data_raw, target_raw = return_labeled_data_color()
data_raw, target_raw = np.array(data_raw)/255, np.array(target_raw)

In [5]:
total_idxs = np.random.permutation(np.arange(0, len(data_raw), 1))
split_val_idx = int(np.floor(len(total_idxs)*0.7))
split_test_idx = int(np.floor(len(total_idxs)*0.8))
train_idxs, val_idxs, test_idxs = total_idxs[:split_val_idx], total_idxs[split_val_idx:split_test_idx], total_idxs[split_test_idx:]

In [6]:
X_train, y_train = data_raw[train_idxs], target_raw[train_idxs] 
X_test, y_test = data_raw[test_idxs], target_raw[test_idxs] 

In [11]:
X, y = [],[]
for i in range(len(X_train)):
    if y_train[i] == 0:
        X.append(X_train[i])
        y.append(y_train[i])

### Utils - Forward and Backprop for Convolution and Maxpooling

##### Convolution - Foward

In [7]:
def iter_image_regions_conv(image, kernel_dim):
    k_w,k_h = kernel_dim, kernel_dim
    i_w,i_h = image.shape[1], image.shape[0]
    for i in range(i_h-k_h+1):
        for j in range(i_w-k_w+1):
            yield image[i:i+k_h, j:j+k_w], i, j

def convolve(image, kernels, kernel_dim):
    imc = image.copy()
    # assume squares for kernel and input
    num_kernels = len(kernels)
    k_dim = kernel_dim
    output_img_dim = len(image[0])-k_dim+1
    
    output = np.zeros((num_kernels,output_img_dim,output_img_dim))
    
    for region, i, j in iter_image_regions_conv(imc, k_dim):
        conv = np.sum(region*kernels, axis=(1,2))
        for f in range(num_kernels):
            output[f,i,j] = conv[f]
            
    return output

In [8]:
def convolve_color(image, kernels, kernel_dim):
    imc = image.copy()
    # assume squares for kernel and input
    num_kernels = len(kernels)
    k_dim = kernel_dim
    output_img_dim = len(image[0])-k_dim+1
    
    output = np.zeros((num_kernels,output_img_dim,output_img_dim))
    
    # image in shape of (x,y,channels)
    # kernels in shape (8, 3, k_dim, k_dim)
    for c in range(3):
        for region, i, j in iter_image_regions_conv(imc[:,:,c], k_dim):
            conv = np.sum(region*kernels[:,c,:,:], axis=(1,2))
            for f in range(num_kernels):
                output[f,i,j] += (1/3)*conv[f]
            
    return output

##### Convolution - Backward

In [9]:
def convolve_backprop(image, dOut, kernel_dim):
    d,h,w = dOut.shape
    dKernels = np.zeros((d, kernel_dim, kernel_dim))
    for img_region, i, j in iter_image_regions_conv(image, kernel_dim):
        for f in range(d):
            dKernels[f,:,:] += dOut[f, i, j]*img_region
    return dKernels

In [10]:
def convolve_backprop_color(image, dOut, kernel_dim):
    d,h,w = dOut.shape
    dKernels = np.zeros((d, 3, kernel_dim, kernel_dim))
    for c in range(3):
        for img_region, i, j in iter_image_regions_conv(image[:,:,c], kernel_dim):
            for f in range(d):
                dKernels[f,c,:,:] += (1/3)*dOut[f, i, j]*img_region
    return dKernels

##### MaxPooling - Foward

In [11]:
def iter_image_regions_pool(image, pool_dim):
    # pool dim will be 2 for this project
    p_w,p_h = pool_dim, pool_dim
    i_w,i_h = image.shape[1], image.shape[0]
    for i in range(i_h//p_h):
        for j in range(i_w//p_w):
            yield image[p_h*i:p_h*i+p_h, p_w*j:p_w*j+p_w], i, j

def max_pool(channels, pool_dim):
    n, img_h, img_w = channels.shape
    output = np.zeros((n, img_h//pool_dim, img_w//pool_dim))
    for c in range(n):
        for region, i, j in iter_image_regions_pool(channels[c], pool_dim):
            max_reg = np.max(region)
            output[c, i, j] = max_reg
    return output

##### MaxPooling - Backward

In [12]:
def max_pool_backprop(channels, dInput, pool_dim):
    d,h,w = channels.shape
    dPool = np.zeros((d, h, w))
    for c in range(d):
        for region, i, j in iter_image_regions_pool(channels[:,:,c], pool_dim):
            # we get the original 2 by 2 region that produced
            max_val = np.max(region)
            inds = np.argwhere(region == max_val)
            dPool[c, 2*i+inds[:,0], 2*j+inds[:,1]] = dInput[c,i,j]
    return dPool

##### Forward Iteration

In [13]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [14]:
def tanh(x) :
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

In [15]:
def loss(y,y_pred):
    epsilon = 0.0001
    log_loss = y*np.log2(y_pred+epsilon) + (1-y)*np.log2(1-y_pred+epsilon)
    return -log_loss

Single Layer Perceptron for Network [since you're not really adding non-linearity]

#### CNN With RELU (only on Linear Layer)

In [ ]:
filters = np.random.rand(8,3,3)/9 # Xavier Initialization for some reason
# w2_dim = linear_input.shape[1]
w2_dim = 2048 
w1_dim = 256
W2 = np.random.rand(w1_dim, w2_dim).astype('float64')*np.sqrt(2/(w2_dim))
W1 = np.random.rand(1, w1_dim).astype('float64')*np.sqrt(2/(w1_dim))
LR = 0.01

In [133]:
for i in range(10):
    avg_loss = []
    for img, y in zip(X_train, y_train):
        
# =============== FORWARD PROPIGATION (WITH RELU) ====================
        # CONVOLUTIONAL LAYER
        output = convolve(img, filters, 3)
        CONV_OUTPUT = output.copy()
        
        # ***<RELU ACTIVATION>***
#         output[output < 0] = 0

        RELU_CONV_OUTPUT = output.copy()
        output = max_pool(output, 2)
        MAXPOOL_OUTPUT = output.copy()
        
        # LINEAR LAYER
        linear_input = output.flatten().reshape((1,-1)) # we're good here if we reconstruct this
        linear_output = linear_input.dot(W2.T)
        W2_OUT = linear_output.copy()
        # SUPER IMPORTANT
        x1 = W2_OUT
        
        # ***<RELU ACTIVATION>***
        linear_output[linear_output < 0] = 0
#         linear_output = sigmoid(linear_output)
#         sig = linear_output.copy()
    
#         print(filters, W1, W2)
#         print(W2_OUT)
        
        linear_output = sigmoid(W1.dot(linear_output.T))
        pred = linear_output
        
# =============== PREDICTION ====================

        print(f'Y_i:{y}, Pred:{pred}')
        yi = y
        avg_loss.append(loss(y, pred))
        
# =============== BACK-PROPIGATION (WITH RELU) ====================
        
        # LINEAR LAYER
        dW1 = (pred-yi).dot(x1)
        
        # ***<RELU GRADIENT> - trying with TanH***
        RELU_GRADIENT = np.ones((1, w1_dim))
        RELU_GRADIENT[W2_OUT < 0 ] = 0
        
        dW2 = ((pred-yi).T.dot(W1)*RELU_GRADIENT).T.dot(linear_input)
        dInput = ((pred-yi).T.dot(W1)*RELU_GRADIENT).dot(W2)

#         dW2 = ((pred-yi).T.dot(W1)).T.dot(linear_input)
#         dInput = ((pred-yi).T.dot(W1)).dot(W2)

#         print(dW1, dW2)
        W1 -= LR*dW1
        W2 -= LR*dW2
        
        # CONVOLUTIONAL LAYER
        IMAGE = img.copy()

        dInput_r = dInput.copy().reshape(MAXPOOL_OUTPUT.shape)
        dOut = max_pool_backprop(RELU_CONV_OUTPUT, dInput_r, 2)
        
        # ***<RELU GRADIENT>***
#         RELU_GRAD_2D = np.ones(CONV_OUTPUT.shape)
#         RELU_GRAD_2D[CONV_OUTPUT < 0] = 0
#         dOut_relu = dOut*RELU_GRAD_2D

        dOut_relu = dOut
        dFilters = convolve_backprop(IMAGE, dOut_relu, 3)
        
        filters -= LR*dFilters
    avg_loss = np.array(avg_loss)
    print(f'EPOCH:{i} - Avg NLLoss: {np.sum(avg_loss)}')

Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:0.0, Pred:[[1.]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:1.0, Pred:[[0.5]]
Y_i:0.0, Pred:[[0.

KeyboardInterrupt: 

### ANN with Activation Functions
- Can try batch later

In [182]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
X, y = mnist["data"], mnist["target"]

In [ ]:
X = X / 255

In [125]:
AW1_dim = 34*34 
AW2_dim = 256
AW1 = np.random.rand(AW2_dim, AW1_dim).astype('float64')
AW2 = np.random.rand(1, AW2_dim).astype('float64')
LR = 0.01

In [126]:
for i in range(10):
    avg_loss = []
    BATCH = 5
    counter = 0
    for img, y in zip(X_train, y_train):
        x = img.reshape((-1, 1))
        out1 = AW1.dot(x)
        out1 = tanh(out1)
        out2 = AW2.dot(out1)
        pred = sigmoid(out2)
        
        yi = y

        avg_loss.append(loss(y,pred))
        dAW2 = (pred-yi).dot(out1.T)
#         dAW1 = ((pred-yi).dot(AW2).T*(out1)*(1-out1)).dot(x.T)
        dAW1 = ((pred-yi).dot(AW2).T*(1-out1**2)).dot(x.T)
        if counter == BATCH:
            counter = 0
            AW1 -= (1/BATCH)*LR*dAW1
            AW2 -= (1/BATCH)*LR*dAW2
        else: 
            counter += 1
    print(f'EPOCH:{i} - Avg NLLoss: {np.sum(np.array(avg_loss))}')
#         dW1 = ((pred-yi).T.dot(AW2)*(out1)*(1-out1)).T.dot(x)

EPOCH:0 - Avg NLLoss: 712.8569813456595
EPOCH:1 - Avg NLLoss: 188.13404224323682
EPOCH:2 - Avg NLLoss: 188.13404224323682
EPOCH:3 - Avg NLLoss: 188.13404224323685
EPOCH:4 - Avg NLLoss: 188.13404224323685
EPOCH:5 - Avg NLLoss: 188.13404224323682
EPOCH:6 - Avg NLLoss: 188.13404224323682
EPOCH:7 - Avg NLLoss: 188.13404224323682
EPOCH:8 - Avg NLLoss: 188.13404224323682
EPOCH:9 - Avg NLLoss: 188.13404224323688


#### CNN Without RELU - WORKING (and what you can go with!)

In [88]:
filters = np.random.rand(8,3,3)/9 # Xavier Initialization for some reason
# w2_dim = linear_input.shape[1]
w2_dim = 2048 
w1_dim = 256
W2 = np.random.rand(w1_dim, w2_dim).astype('float64')*np.sqrt(2/(w2_dim))
W1 = np.random.rand(1, w1_dim).astype('float64')*np.sqrt(2/(w1_dim))
LR = 0.01

In [89]:
for i in range(10):
    avg_loss = []
    for img, y in zip(X_train, y_train):
        
# =============== FORWARD PROPIGATION (WITH RELU) ====================
        # CONVOLUTIONAL LAYER
        output = convolve(img, filters, 3)
        CONV_OUTPUT = output.copy()
        
        # ***<RELU ACTIVATION>***
#         output[output < 0] = 0

        RELU_CONV_OUTPUT = output.copy()
        output = max_pool(output, 2)
        MAXPOOL_OUTPUT = output.copy()
        
        # LINEAR LAYER
        linear_input = output.flatten().reshape((1,-1)) # we're good here if we reconstruct this
        linear_output = linear_input.dot(W2.T)
        W2_OUT = linear_output.copy()
        
        # ***<RELU ACTIVATION>***
#         linear_output[linear_output < 0] = 0
        x1 = W2_OUT
        linear_output = sigmoid(W1.dot(linear_output.T))
        pred = linear_output
        
# =============== PREDICTION ====================

        print(f'Y_i:{y}, Pred:{pred}')
        yi = y
        avg_loss.append(loss(y, pred))
        
# =============== BACK-PROPIGATION (WITH RELU) ====================
        
        # LINEAR LAYER
        dW1 = (pred-yi).dot(x1)
        
        # ***<RELU GRADIENT>***
#         RELU_GRADIENT = np.ones((1, w1_dim))
#         RELU_GRADIENT[W2_OUT < 0 ] = 0
        
#         dW2 = ((pred-yi).T.dot(W1)*RELU_GRADIENT).T.dot(linear_input)
#         dInput = ((pred-yi).T.dot(W1)*RELU_GRADIENT).dot(W2)

        dW2 = ((pred-yi).T.dot(W1)).T.dot(linear_input)
        dInput = ((pred-yi).T.dot(W1)).dot(W2)

        W1 -= LR*dW1
        W2 -= LR*dW2
        
        # CONVOLUTIONAL LAYER
        IMAGE = img.copy()

        dInput_r = dInput.copy().reshape(MAXPOOL_OUTPUT.shape)
        dOut = max_pool_backprop(RELU_CONV_OUTPUT, dInput_r, 2)
        
        # ***<RELU GRADIENT>***
#         RELU_GRAD_2D = np.ones(CONV_OUTPUT.shape)
#         RELU_GRAD_2D[CONV_OUTPUT < 0] = 0
#         dOut_relu = dOut*RELU_GRAD_2D

        dOut_relu = dOut
#         print(dOut_relu)
        dFilters = convolve_backprop(IMAGE, dOut_relu, 3)
        
        filters -= LR*dFilters
    avg_loss = np.array(avg_loss)
    print(f'EPOCH:{i} - Avg NLLoss: {np.sum(avg_loss)}')

Y_i:1.0, Pred:[[1.]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0

Y_i:1.0, Pred:[[1.]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0

<ipython-input-45-3cbdbdecc9c7>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Y_i:1.0, Pred:[[0.]]
[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [-0.29180668  0.         -0.31311938 ... -0.32341968  0.
   -0.31634282]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [-0.31017837  0.         -0.32255554 ... -0.30665563  0.
   -0.31134189]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    

Y_i:1.0, Pred:[[1.]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0

...

Y_i:1.0, Pred:[[0.99984132]]
[[[0.00000000e+00 8.66263325e-06 0.00000000e+00 ... 0.00000000e+00
   7.89933197e-06 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 7.53331133e-06 0.00000000e+00 ... 0.00000000e+00
   1.26970469e-05 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 4.02440028e-06 0.00000000e+00 ... 0.00000000e+00
   9.05703091e-06 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 2.30128050e-06 0.00000000e+00 ... 0.00000000e+00
   9.33929023e-06 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 

Y_i:0.0, Pred:[[0.99889675]]
[[[ 0.         -0.054533    0.         ...  0.         -0.04972811
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.         -0.04742352  0.         ...  0.         -0.07993109
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.         -0.02533405  0.         ...  0.         -0.05701618
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.         -0.01448644  0.         ...  0.         -0.05879306
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.         -0.02803

Y_i:1.0, Pred:[[0.99994093]]
[[[0.00000000e+00 2.83483724e-06 0.00000000e+00 ... 0.00000000e+00
   2.57676349e-06 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 2.43354574e-06 0.00000000e+00 ... 0.00000000e+00
   4.26183937e-06 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[0.00000000e+00 1.17901376e-06 0.00000000e+00 ... 0.00000000e+00
   2.99170777e-06 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 5.61864742e-07 0.00000000e+00 ... 0.00000000e+00
   3.06589352e-06 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 

   0.00000000e+00 0.00000000e+00]]]
Y_i:0.0, Pred:[[7.52459949e-05]]
[[[ 0.00000000e+00 -3.84981625e-06  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   -3.53964978e-06  0.00000000e+00]
  [ 0.00000000e+00 -3.33953679e-06  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00 -1.73525842e-06  0.00000000e+00 ...  0.00000000e+00
   -4.06623792e-06  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -9.50928789e-07  0.00000000e+00 ...  0.00000000e

KeyboardInterrupt: 

### Working CNN with Color

In [16]:
color_filters = np.random.rand(8,3,3,3)
# w2_dim = linear_input.shape[1]
w2_dim = 2048 
w1_dim = 256
W2 = np.random.rand(w1_dim, w2_dim).astype('float64')*np.sqrt(2/(w2_dim))
W1 = np.random.rand(1, w1_dim).astype('float64')*np.sqrt(2/(w1_dim))
LR = 0.01

In [17]:
for i in range(10):
    avg_loss = []
    for img, y in zip(X_train, y_train):
        
# =============== FORWARD PROPIGATION (WITH RELU) ====================
        # CONVOLUTIONAL LAYER
        output = convolve_color(img, color_filters, 3)
        CONV_OUTPUT = output.copy()
        RELU_CONV_OUTPUT = output.copy()
        output = max_pool(output, 2)
        MAXPOOL_OUTPUT = output.copy()
        
        # LINEAR LAYER
        linear_input = output.flatten().reshape((1,-1)) # we're good here if we reconstruct this
        linear_output = linear_input.dot(W2.T)
        W2_OUT = linear_output.copy()
        
        x1 = W2_OUT
        linear_output = sigmoid(W1.dot(linear_output.T))
        pred = linear_output
        
        
# =============== PREDICTION ====================

        print(f'Y_i:{y}, Pred:{pred}')
        yi = y
        avg_loss.append(loss(y, pred))
        
# =============== BACK-PROPIGATION (WITH RELU) ====================
        
        # LINEAR LAYER
        dW1 = (pred-yi).dot(x1)

        dW2 = ((pred-yi).T.dot(W1)).T.dot(linear_input)
        dInput = ((pred-yi).T.dot(W1)).dot(W2)

        W1 -= LR*dW1
        W2 -= LR*dW2
        
        # CONVOLUTIONAL LAYER
        IMAGE = img.copy()

        dInput_r = dInput.copy().reshape(MAXPOOL_OUTPUT.shape)
        dOut = max_pool_backprop(RELU_CONV_OUTPUT, dInput_r, 2)
        
        dOut_relu = dOut

        dFilters = convolve_backprop_color(IMAGE, dOut_relu, 3)
    
        color_filters -= LR*dFilters
        
    avg_loss = np.array(avg_loss)
    print(f'EPOCH:{i} - Avg NLLoss: {np.sum(avg_loss)}')

Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:1.0, Pred:[[1.]]
Y_i:0.0, Pred:[[1.]]


<ipython-input-13-3cbdbdecc9c7>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Y_i:0.0, Pred:[[0.]]
Y_i:0.0, Pred:[[0.]]
Y_i:0.0, Pred:[[0.]]
Y_i:1.0, Pred:[[0.]]
Y_i:0.0, Pred:[[1.]]
Y_i:0.0, Pred:[[1.31502598e-49]]
Y_i:0.0, Pred:[[1.70143472e-46]]
Y_i:1.0, Pred:[[2.26886353e-53]]
Y_i:1.0, Pred:[[0.45994772]]
Y_i:1.0, Pred:[[0.76291554]]
Y_i:0.0, Pred:[[0.81082685]]
Y_i:0.0, Pred:[[0.47343412]]
Y_i:0.0, Pred:[[0.40410676]]
Y_i:0.0, Pred:[[0.31241739]]
Y_i:1.0, Pred:[[0.29283932]]
Y_i:1.0, Pred:[[0.43400765]]
Y_i:0.0, Pred:[[0.4696889]]
Y_i:0.0, Pred:[[0.37342988]]
Y_i:1.0, Pred:[[0.38292434]]
Y_i:0.0, Pred:[[0.41584401]]
Y_i:0.0, Pred:[[0.3493813]]
Y_i:1.0, Pred:[[0.32735769]]
Y_i:1.0, Pred:[[0.46498587]]
Y_i:1.0, Pred:[[0.58845509]]
Y_i:1.0, Pred:[[0.67331909]]
Y_i:0.0, Pred:[[0.70187774]]
Y_i:0.0, Pred:[[0.48170415]]
Y_i:1.0, Pred:[[0.50590998]]
Y_i:0.0, Pred:[[0.45219517]]
Y_i:0.0, Pred:[[0.40825984]]
Y_i:1.0, Pred:[[0.43981746]]
Y_i:1.0, Pred:[[0.54537826]]
Y_i:0.0, Pred:[[0.43533543]]
Y_i:1.0, Pred:[[0.57718796]]
Y_i:1.0, Pred:[[0.70416114]]
Y_i:1.0, Pred:[

Y_i:1.0, Pred:[[0.97838721]]
Y_i:0.0, Pred:[[0.21056685]]
Y_i:0.0, Pred:[[0.22863739]]
Y_i:0.0, Pred:[[0.00861923]]
Y_i:0.0, Pred:[[0.00939678]]
Y_i:0.0, Pred:[[0.00924723]]
EPOCH:1 - Avg NLLoss: 53.751191894806425
Y_i:1.0, Pred:[[0.60974501]]
Y_i:1.0, Pred:[[0.96634965]]
Y_i:1.0, Pred:[[0.9869398]]
Y_i:0.0, Pred:[[0.26844353]]
Y_i:0.0, Pred:[[0.02853524]]
Y_i:0.0, Pred:[[0.03592005]]
Y_i:0.0, Pred:[[0.01384533]]
Y_i:1.0, Pred:[[0.84968164]]
Y_i:0.0, Pred:[[0.0342165]]
Y_i:0.0, Pred:[[0.04752193]]
Y_i:0.0, Pred:[[0.04349149]]
Y_i:1.0, Pred:[[0.80916312]]
Y_i:1.0, Pred:[[0.94730509]]
Y_i:1.0, Pred:[[0.96935571]]
Y_i:0.0, Pred:[[0.07065835]]
Y_i:0.0, Pred:[[0.15657746]]
Y_i:0.0, Pred:[[0.00870325]]
Y_i:0.0, Pred:[[0.01449253]]
Y_i:1.0, Pred:[[0.91790353]]
Y_i:1.0, Pred:[[0.93486862]]
Y_i:0.0, Pred:[[0.04521268]]
Y_i:0.0, Pred:[[0.09570492]]
Y_i:1.0, Pred:[[0.86761073]]
Y_i:0.0, Pred:[[0.0394893]]
Y_i:0.0, Pred:[[0.13341593]]
Y_i:1.0, Pred:[[0.92934687]]
Y_i:1.0, Pred:[[0.91638027]]
Y_i:1

Y_i:0.0, Pred:[[0.17143358]]
Y_i:0.0, Pred:[[0.07906663]]
Y_i:0.0, Pred:[[0.02153418]]
Y_i:0.0, Pred:[[0.00090313]]
Y_i:1.0, Pred:[[0.92252018]]
Y_i:1.0, Pred:[[0.98960214]]
Y_i:1.0, Pred:[[0.96019592]]
Y_i:1.0, Pred:[[0.99802024]]
Y_i:1.0, Pred:[[0.980575]]
Y_i:0.0, Pred:[[0.01091705]]
Y_i:1.0, Pred:[[0.99666316]]
Y_i:1.0, Pred:[[0.99534638]]
Y_i:1.0, Pred:[[0.99863858]]
Y_i:1.0, Pred:[[0.99739606]]
Y_i:0.0, Pred:[[0.02493625]]
Y_i:0.0, Pred:[[0.17569151]]
Y_i:0.0, Pred:[[0.00045277]]
Y_i:0.0, Pred:[[0.00051243]]
Y_i:0.0, Pred:[[0.00056094]]
EPOCH:3 - Avg NLLoss: 6.3166425504645645
Y_i:1.0, Pred:[[0.89858428]]
Y_i:1.0, Pred:[[0.98209576]]
Y_i:1.0, Pred:[[0.99656011]]
Y_i:0.0, Pred:[[0.02021298]]
Y_i:0.0, Pred:[[0.00419136]]
Y_i:0.0, Pred:[[0.0079374]]
Y_i:0.0, Pred:[[0.00188903]]
Y_i:1.0, Pred:[[0.99040746]]
Y_i:0.0, Pred:[[0.00254257]]
Y_i:0.0, Pred:[[0.00631861]]
Y_i:0.0, Pred:[[0.00731478]]
Y_i:1.0, Pred:[[0.96985463]]
Y_i:1.0, Pred:[[0.98843783]]
Y_i:1.0, Pred:[[0.98962377]]
Y_i:0

Y_i:1.0, Pred:[[0.98991383]]
Y_i:1.0, Pred:[[0.99972745]]
Y_i:1.0, Pred:[[0.99867052]]
Y_i:0.0, Pred:[[0.00330264]]
Y_i:0.0, Pred:[[0.01201887]]
Y_i:1.0, Pred:[[0.99944394]]
Y_i:0.0, Pred:[[0.00278691]]
Y_i:0.0, Pred:[[0.00266959]]
Y_i:0.0, Pred:[[0.00572275]]
Y_i:0.0, Pred:[[0.0034954]]
Y_i:0.0, Pred:[[0.00065766]]
Y_i:1.0, Pred:[[0.96102757]]
Y_i:1.0, Pred:[[0.90722336]]
Y_i:0.0, Pred:[[0.08327676]]
Y_i:0.0, Pred:[[0.10375622]]
Y_i:0.0, Pred:[[0.01159927]]
Y_i:0.0, Pred:[[0.00031074]]
Y_i:1.0, Pred:[[0.97281016]]
Y_i:1.0, Pred:[[0.99402736]]
Y_i:1.0, Pred:[[0.96361747]]
Y_i:1.0, Pred:[[0.99912869]]
Y_i:1.0, Pred:[[0.98475487]]
Y_i:0.0, Pred:[[0.00268791]]
Y_i:1.0, Pred:[[0.99843218]]
Y_i:1.0, Pred:[[0.9977591]]
Y_i:1.0, Pred:[[0.99942246]]
Y_i:1.0, Pred:[[0.99874848]]
Y_i:0.0, Pred:[[0.00711484]]
Y_i:0.0, Pred:[[0.10031041]]
Y_i:0.0, Pred:[[0.00025344]]
Y_i:0.0, Pred:[[0.00029167]]
Y_i:0.0, Pred:[[0.00033129]]
EPOCH:5 - Avg NLLoss: 2.734597539050352
Y_i:1.0, Pred:[[0.97662224]]
Y_i:1

Y_i:0.0, Pred:[[0.00024268]]
Y_i:0.0, Pred:[[0.00531185]]
Y_i:1.0, Pred:[[0.96687137]]
Y_i:1.0, Pred:[[0.97025788]]
Y_i:1.0, Pred:[[0.9996695]]
Y_i:1.0, Pred:[[0.99967192]]
Y_i:1.0, Pred:[[0.99791306]]
Y_i:0.0, Pred:[[0.0013674]]
Y_i:1.0, Pred:[[0.99955256]]
Y_i:1.0, Pred:[[0.99878084]]
Y_i:0.0, Pred:[[0.00147157]]
Y_i:1.0, Pred:[[0.99929558]]
Y_i:0.0, Pred:[[0.00072132]]
Y_i:1.0, Pred:[[0.99182104]]
Y_i:1.0, Pred:[[0.99985082]]
Y_i:1.0, Pred:[[0.99914986]]
Y_i:0.0, Pred:[[0.00131163]]
Y_i:0.0, Pred:[[0.00532148]]
Y_i:1.0, Pred:[[0.9997248]]
Y_i:0.0, Pred:[[0.00124108]]
Y_i:0.0, Pred:[[0.00124009]]
Y_i:0.0, Pred:[[0.00288127]]
Y_i:0.0, Pred:[[0.00185775]]
Y_i:0.0, Pred:[[0.00030553]]
Y_i:1.0, Pred:[[0.97341692]]
Y_i:1.0, Pred:[[0.91554406]]
Y_i:0.0, Pred:[[0.04914408]]
Y_i:0.0, Pred:[[0.11241462]]
Y_i:0.0, Pred:[[0.0071454]]
Y_i:0.0, Pred:[[0.00014869]]
Y_i:1.0, Pred:[[0.98571022]]
Y_i:1.0, Pred:[[0.99619903]]
Y_i:1.0, Pred:[[0.96861037]]
Y_i:1.0, Pred:[[0.99948584]]
Y_i:1.0, Pred:[[0.

Y_i:1.0, Pred:[[0.96545247]]
Y_i:1.0, Pred:[[0.99971623]]
Y_i:1.0, Pred:[[0.9999435]]
Y_i:0.0, Pred:[[0.00597834]]
Y_i:0.0, Pred:[[0.00841404]]
Y_i:0.0, Pred:[[0.00143211]]
Y_i:0.0, Pred:[[0.00992079]]
Y_i:1.0, Pred:[[0.99987687]]
Y_i:0.0, Pred:[[0.00025877]]
Y_i:0.0, Pred:[[0.00031846]]
Y_i:0.0, Pred:[[0.00175474]]
Y_i:0.0, Pred:[[0.00098327]]
Y_i:0.0, Pred:[[0.05300787]]
Y_i:0.0, Pred:[[0.00017939]]
Y_i:0.0, Pred:[[0.00455309]]
Y_i:1.0, Pred:[[0.98191834]]
Y_i:1.0, Pred:[[0.97612643]]
Y_i:1.0, Pred:[[0.99977945]]
Y_i:1.0, Pred:[[0.99978006]]
Y_i:1.0, Pred:[[0.99839]]
Y_i:0.0, Pred:[[0.00068814]]
Y_i:1.0, Pred:[[0.99968517]]
Y_i:1.0, Pred:[[0.99909801]]
Y_i:0.0, Pred:[[0.00075095]]
Y_i:1.0, Pred:[[0.99950109]]
Y_i:0.0, Pred:[[0.00036987]]
Y_i:1.0, Pred:[[0.99307665]]
Y_i:1.0, Pred:[[0.9999044]]
Y_i:1.0, Pred:[[0.99939235]]
Y_i:0.0, Pred:[[0.00068196]]
Y_i:0.0, Pred:[[0.00294103]]
Y_i:1.0, Pred:[[0.99982715]]
Y_i:0.0, Pred:[[0.00065719]]
Y_i:0.0, Pred:[[0.00066979]]
Y_i:0.0, Pred:[[0.0

### END IMPORTANT SEGMENT

#### Backprop - Linear

In [279]:
yi = 0
dW1 = (pred-yi).dot(x1)
RELU_GRAD = np.ones((1, w1_dim))
RELU_GRAD[raw_x1 < 0 ] = 0

dW2 = ((pred-yi).T.dot(W1)*RELU_GRAD).T.dot(linear_input)
dInput = ((pred-yi).T.dot(W1)*RELU_GRAD).dot(W2)

In [280]:
W1 -= LR*dW1
W2 -= LR*dW2

#### Backprop - Conv

In [281]:
RELU_CONV_OUTPUT = relu_conv_output.copy()
CONV_OUTPUT = conv_output.copy()
IMAGE = img_test_train.copy()

dInput_r = dInput.copy().reshape(max_pool_output.shape)
dOut = max_pool_backprop(RELU_CONV_OUTPUT, dInput_r, 2)
RELU_GRAD_2D = np.ones(CONV_OUTPUT.shape)
RELU_GRAD_2D[CONV_OUTPUT < 0] = 0
dOut_relu = dOut*RELU_GRAD_2D
dFilters = convolve_backprop(IMAGE, dOut_relu, 3)

In [282]:
filters -= LR*dFilters

### Testing
- CONVOLUTION IS VALID FROM WHAT I CAN TELL
- MAX POOL IS VALID FROM WHAT I CANT TELL

In [84]:
filters = np.zeros((8,3,3))
output1 = np.zeros((8,32,32))

In [91]:
filters = np.random.rand(2,2,2)
# print(filters, np.sum(filters, axis=(1,2)))
# filters = np.ones((2,2,2))
img_conv_test = np.array([[1,1,1],[1,1,1],[1,1,1]])
convolve(img_conv_test, filters, 2)

array([[[1.99173722, 1.99173722],
        [1.99173722, 1.99173722]],

       [[1.75918869, 1.75918869],
        [1.75918869, 1.75918869]]])

In [104]:
img_pool_dim_test = np.array([[1,1,2,2],[1,1,2,2],[3,3,4,4],[3,3,4,4]])
img_pool_test = np.array([img_pool_dim_test,img_pool_dim_test*2])
img_big_pool_dim_test = np.ones((32,32))
img_big_pool_test = np.array([img_big_pool_dim_test,img_big_pool_dim_test*2])

img_channels_test = np.ones((2,4,4))
count = 0
# TESTING POOL REGIONS CORRECT
# for region,i,j in iter_image_regions_pool(img_channels_test, 2):
#     print(region, i, j)
#     count += 1
# print(count)

# TEST POOL CHANNEL CREATION
max_pool(img_big_pool_test, 2)

array([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1

### Full Class

### Junk Work

In [19]:
a = np.array([[[0,1],[2,3]],[[9,5],[-8,10]]])

In [20]:
a

array([[[ 0,  1],
        [ 2,  3]],

       [[ 9,  5],
        [-8, 10]]])

In [21]:
a[a < 0] = 0
a

array([[[ 0,  1],
        [ 2,  3]],

       [[ 9,  5],
        [ 0, 10]]])

In [14]:
h, w, f = a.shape
for i in range(h):
    for j in range(w):
        for fi in range(f):
            print(a[i,j,fi])

0
1
2
3
9
5
8
10


In [17]:
a[0, 1,0]

2

In [11]:
b=a.flatten()
print(b)
c=b.reshape((a.shape))
c

[ 0  1  2  3  9  5  8 10]


array([[[ 0,  1],
        [ 2,  3]],

       [[ 9,  5],
        [ 8, 10]]])

In [7]:
a = np.array([[-2,1],[2,-3]])
a[a < 0] = 0

In [8]:
a

array([[0, 1],
       [2, 0]])

In [24]:
np.zeros((8,3,3))[0]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [28]:
a = np.array([[1,0],[0,1]])
A = np.array([a,a])
b = np.array([[[1,2],[1,2]],[[3,4],[1,2]],[[5,6],[1,2]]])
e = np.array([[[5,2],[1,2]],[[7,4],[10,2]],[[5,6],[1,2]]])
c = np.array([b,e,b])
print(np.sum(A*c, axis=(1,2,3)))
d = A*c
print(d)

ValueError: operands could not be broadcast together with shapes (2,2,2) (3,3,2,2) 

In [14]:
a = np.array([[1,0],[0,1]])
b = np.array([[[1,2],[1,2]],[[3,4],[1,2]],[[5,6],[1,2]]])
print(np.sum(a*b, axis=(1,2)))
c = a*b
print(c)

[3 5 7]
[[[1 0]
  [0 2]]

 [[3 0]
  [0 2]]

 [[5 0]
  [0 2]]]


In [98]:
np.max(b)

6

In [30]:
filters = np.random.rand(8,3,3,3)/9 # Xavier Initialization for some reason
filters[:,0,:,:]

array([[[0.109733  , 0.0609449 , 0.09934462],
        [0.00475399, 0.11082625, 0.03005487],
        [0.00593952, 0.02024118, 0.07247198]],

       [[0.03760646, 0.0382526 , 0.09518495],
        [0.02229376, 0.04821857, 0.03266446],
        [0.04924374, 0.1052221 , 0.05982489]],

       [[0.10920898, 0.03772255, 0.04169418],
        [0.06990995, 0.06279942, 0.07886027],
        [0.01181489, 0.00124126, 0.10478992]],

       [[0.06106537, 0.06161425, 0.06452405],
        [0.02691429, 0.0353332 , 0.10398888],
        [0.11012785, 0.01257094, 0.04778741]],

       [[0.06380113, 0.07184055, 0.07311916],
        [0.06231061, 0.06829288, 0.07829647],
        [0.05933732, 0.03077207, 0.01799888]],

       [[0.07076081, 0.07140797, 0.06357909],
        [0.02985641, 0.09051258, 0.0362897 ],
        [0.04613158, 0.05256429, 0.07955669]],

       [[0.07256847, 0.04868344, 0.09700728],
        [0.02700287, 0.04991762, 0.07407325],
        [0.05676463, 0.0553604 , 0.04743678]],

       [[0.01040914,